In [1]:
# Generic
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings, gc
warnings.filterwarnings("ignore")

# Tensorflow
import tensorflow as tf

# ktrain
import ktrain
from ktrain import text

# sklearn
from sklearn.model_selection import train_test_split


In [2]:

df = pd.read_csv("TrainingDataForBERTMODEL15-9-Delivery-Binary.csv", header='infer')
# Dropping Null Values
df.dropna(inplace=True)

# Total Records
print("Total Records: ", df.shape[0])

# Inspect
df.head()


Total Records:  2038


,Topic,Sentence
0,No,Begins with a story to engage audience
1,Delivery,He clasps his hands a lot while he speaks
2,Delivery,Use of humour
3,Delivery,Use of analogy
4,Delivery,He makes good eye contact


In [3]:
# Data Split
target = ['Topic']
data = ['Sentence']

X = df[data]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=42)

In [4]:
# Common Parameters
max_len = 500
batch_size = 16
learning_rate = 5e-5
epochs = 5

In [5]:
#########
## With Transformer = bert-base-uncased
######
# Transformer Model
model_ = 'bert-base-uncased'
t_mod = text.Transformer(model_, maxlen=500, classes = [0,1])


'''Converting split data to list [so it can processed]'''
#train
X_tr = X_train['Sentence'].tolist()
y_tr = y_train['Topic'].tolist()

#test
X_ts = X_test['Sentence'].tolist()
y_ts = y_test['Topic'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=batch_size)

preprocessing train...
language: en
train sequence lengths:
	mean : 11
	95percentile : 28
	99percentile : 52


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 25
	99percentile : 54


Is Multi-Label? False


In [ ]:
# Train Model
learner.fit_onecycle(learning_rate, epochs)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/5
 22/115 [====>.........................] - ETA: 2:04:07 - loss: 0.6756 - accuracy: 0.5966

In [7]:
# Evaluate
x = learner.validate(class_names=t_mod.get_classes())

              precision    recall  f1-score   support

    Delivery       0.94      0.89      0.91       123
          No       0.84      0.91      0.88        81

    accuracy                           0.90       204
   macro avg       0.89      0.90      0.89       204
weighted avg       0.90      0.90      0.90       204



In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)
predictor.save('DeliveryBinaryClassifier')
print('Model Saved')

In [8]:
model.save('completeDeliv_saved_model/')

In [1]:
df1=pd.read_csv("PresVideosTo6sentencesusingFullStopasDelimiter.csv")
#classes = ['Delivery','Structure','Visual Aids']
classes = ['Delivery','']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)

segmentId=df1['SegmentId']
segList=list(segmentId)

startTime=df1['StartTime']
strList=list(startTime)

endTime=df1['EndTime']
endList=list(endTime)

transcript=df1['Transcript']
transList=list(transcript)
transList1=[]
for i in transList:
    if i != 'nan':
        transList1.append(i)



NameError: name 'pd' is not defined

In [ ]:
i=0
segTranscript=[]
allSegmentsTrans=[]
while i < len(segList)-2:
    #print(segList[i])
    #print()
    if segList[i]==segList[i+1]:
        print(segList[i],'  ==  ',segList[i+1])
        print()
        segTranscript.append(transList1[i])
        #segTranscript.append(transList1[i+1])
        print( transList1[i])
        #print( transList1[i+1])
        print()
        i+=1
    else:
        print(segList[i],'  !=  ',segList[i+1])
        print()
        
        segTranscript.append(transList1[i])
        listToString=' '.join([str(item) for item in segTranscript ])
        allSegmentsTrans.append([segList[i],strList[i-5],endList[i-5],segTranscript,listToString])
        print(listToString)
        print()
        segTranscript=[]
        i+=1

In [1]:
#### 
# Write the annotation result to Excel File

import xlrd
import xlsxwriter
workbook = xlsxwriter.Workbook('PresentationCorpusAnnotatedUsingBertBaseUncase-BinaryD.xlsx')
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
worksheet.write('E1', 'Topic')
row=1
col=0


for i in allSegmentsTrans:
    
    worksheet.write(row,col,i[0])
    worksheet.write(row,col+1,i[1])
    worksheet.write(row,col+2,i[2])
    transRow=row
    for j in i[3]:
        worksheet.write(transRow,col+3,j)
        transRow+=1
    worksheet.write(row,col+4,predictor.predict(i[4]))
        
        

    row+=transRow  
workbook.close()        

#pred_class = predictor.predict(transList )
#print("Predicted Class: ", predictor.predict(transList ))


NameError: name 'transList1' is not defined